# **ETL**

Descarga de librerias

In [ ]:
import pandas as pd
import numpy as np
import json
import re
pd.set_option('display.max_columns', None) 


LESTURA DEL DATASET CREDITS

In [ ]:
credit = pd.read_csv('credits.csv',encoding='utf-8')
print(np.shape(credit))
credit.head(2)


In [ ]:
credit.rename(columns={'id': 'id_pelicula'}, inplace=True)

In [ ]:
credit.count()

LIMPIEZA DE LOS DATOS 


Este Dataframe CREDITS, aparentaba tener valores anidados en formato JSON, pero resultaron ser cadenas de texto. Por tal motivo, se procede a crear esta funcion, para iterar cada uno de los registro, eliminando espacios vacios y eliminando caracteres especiales, dejando como resutado un dataframe mas accesible a desanidar en la sigueinte funcion.

In [ ]:
def limpiar_columnas(df, columnas):
    for col in columnas:
        # Itera sobre cada fila del DataFrame
        for index, row in df.iterrows():
            valor = row[col]
            
            if isinstance(valor, str):
                # Separa la cadena por comas si es una cadena de texto
                items = valor.split(',')
                cleaned_items = []

                for item in items:
                    # Elimina todo lo que no sea palabras, números o el carácter ':'
                    cleaned_item = re.sub(r'[^\w\s:]', '', item)
                    # Corrige espacios vacíos (eliminar espacios adicionales)
                    cleaned_item = cleaned_item.strip()

                    # Añade el elemento limpio a la lista si no está vacío
                    if cleaned_item:  
                        cleaned_items.append(cleaned_item)

                # Actualiza la columna con los elementos limpios
                df.at[index, col] = ', '.join(cleaned_items)
                print(f"Fila {index}, Columna {col}, Elementos limpios: {', '.join(cleaned_items)}")
            else:
                # Si el valor no es una cadena de texto, lo deja sin cambios
                print(f"Fila {index}, Columna {col}, Valor no procesado (tipo {type(valor).__name__}): {valor}")

    # Devuelve el DataFrame con todas las columnas, incluyendo las numéricas
    return df

In [ ]:
columnas_a_limpiar = ['cast', 'crew']
df2 = limpiar_columnas(credit, columnas_a_limpiar)


In [ ]:
df2.head(2)

In [ ]:
df2.count()

Se crea una funcion para reorganizar el Dataframe. 

Esta funcion recibe un Dataframe, una variable de tipo lista, que contiene las columnas a modificar y otra variable tambien tipo lista que contiene los nombres de las columnas nuevas, extraidas del dataframe 

In [ ]:
def desanidar_ycrear_columnas(df, columnas, palabras_clave, columna_int):
    # Diccionario para almacenar las nuevas columnas
    columnas_nuevas = {clave: [] for clave in palabras_clave}
    columna_int_values = []

    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        # Obtener el valor de la columna de tipo int
        valor_int = row[columna_int] if columna_int in df.columns else None
        
        # Variable para almacenar las filas desanidadas en la iteración actual
        filas_desanidadas = []
        
        # Iterar sobre las columnas que contienen datos anidados
        for col in columnas:
            # Verificar si el valor es una cadena, de lo contrario, continuar con la siguiente fila
            if isinstance(row[col], str):
                # Separar la cadena por comas
                items = row[col].split(',')
                
                # Diccionario temporal para almacenar los valores de cada fila
                temp_dict = {clave: None for clave in palabras_clave}
                
                for item in items:
                    # Buscar coincidencias de las palabras clave seguidas del carácter ':'
                    for clave in palabras_clave:
                        if clave in item:
                            # Extraer el valor que sigue al carácter ':'
                            valor = re.search(rf'{clave}\s*:\s*(.*)', item)
                            if valor:
                                temp_dict[clave] = valor.group(1).strip()
                
                # Añadir los valores desanidados a la lista de filas desanidadas
                filas_desanidadas.append(temp_dict)
            else:
                # Si el valor no es una cadena, agregar None a las listas correspondientes
                for clave in palabras_clave:
                    columnas_nuevas[clave].append(None)
                # Agregar el valor de la columna de tipo int a la lista correspondiente
                columna_int_values.append(valor_int)
                
        # Añadir las filas desanidadas a las listas de columnas nuevas y valores int
        for fila in filas_desanidadas:
            for clave in palabras_clave:
                columnas_nuevas[clave].append(fila[clave])
            columna_int_values.append(valor_int)

    # Crear un nuevo DataFrame con las nuevas columnas
    df_nuevo = pd.DataFrame(columnas_nuevas)
    df_nuevo[columna_int] = columna_int_values
    
    return df_nuevo

COLUMNA 'CAST' DESANIDADA Y TRANSFORMADA EN DATAFRAME

In [ ]:
palabras_clave = ['cast_id', 'character','credit_id','gender','id','name','order','profile_path']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['cast']
cast = desanidar_ycrear_columnas(df2, columnas_a_desanidar, palabras_clave,columna_int='id_pelicula')

# Mostrar el nuevo DataFrame
cast.head(2)

In [ ]:
cast.count()

CALUMNA 'CREW' DESANIDADA Y TRANSFORMADA EN DATAFRAME

In [ ]:
palabras_clave = ['credit_id','department','gender','id','job','name','profile_path']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['crew']
crew = desanidar_ycrear_columnas(credit, columnas_a_desanidar, palabras_clave,columna_int='id_pelicula')

# Mostrar el nuevo DataFrame
crew.head(2)

In [ ]:
crew.count()

__________________________________________________________________________________________________________________________________________________________________________________

**LECTURA DEL DATASET MOVIES**

In [ ]:
movies = pd.read_csv('movies_dataset.csv',encoding='utf-8')
print(np.shape(movies))

In [ ]:
movies.head(2)

In [ ]:
# Supongamos que 'df' es tu DataFrame
movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)

In [ ]:
# c. Eliminar los valores nulos del campo 'release_date'
movies = movies.dropna(subset=['release_date'])

In [ ]:
# d. Convertir 'release_date' al formato AAAA-mm-dd y crear la columna 'release_year'
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')
movies['release_year'] = movies['release_date'].dt.year


In [ ]:
# Convertir las columnas 'revenue' y 'budget' a numéricas, reemplazando errores con 0
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce').fillna(0)
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce').fillna(0)

# Crear la columna 'return' con el cálculo de revenue / budget, asignando 0 cuando el presupuesto es 0
movies['return'] = movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

In [ ]:
# Revisión final de las columnas afectadas
movies.head(2)

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
def detectar_valores_complejos(df):
    columnas_complejas = {}
    
    for columna in df.columns:
        tipos_detectados = set()
        
        for valor in df[columna]:
            if isinstance(valor, str):
                try:
                    # Intentar cargar el valor como JSON
                    json_valor = json.loads(valor)
                    if isinstance(json_valor, dict):
                        tipos_detectados.add('JSON (diccionario)')
                    elif isinstance(json_valor, list):
                        tipos_detectados.add('JSON (lista)')
                    else:
                        tipos_detectados.add('JSON (otro)')
                except (ValueError, TypeError):
                    tipos_detectados.add('String (no JSON)')
            elif isinstance(valor, list):
                tipos_detectados.add('Lista')
            elif isinstance(valor, dict):
                tipos_detectados.add('Diccionario')
            elif isinstance(valor, (np.ndarray, pd.Series, pd.DataFrame)):
                tipos_detectados.add(f'{type(valor).__name__}')
            elif isinstance(valor, (set, tuple)):
                tipos_detectados.add(f'{type(valor).__name__}')
            elif isinstance(valor, complex):
                tipos_detectados.add('Número complejo')
        
        if tipos_detectados:
            columnas_complejas[columna] = tipos_detectados
    
    return columnas_complejas

In [ ]:
columnas_complejas = detectar_valores_complejos(movies)
for columna, tipos in columnas_complejas.items():
    print(f"Columna       '{columna}'                             contiene los siguientes tipos de valores:                                             {', '.join(tipos)}")

In [ ]:
def detectar_columnas_anidadas(df):
    columnas_anidadas = {}
    
    for columna in df.columns:
        tipo_anidado = None
        
        for valor in df[columna]:
            if isinstance(valor, str):
                try:
                    # Intentar cargar el valor como JSON
                    json_valor = json.loads(valor)
                    if isinstance(json_valor, dict):
                        tipo_anidado = 'JSON (diccionario)'
                        break
                    elif isinstance(json_valor, list):
                        tipo_anidado = 'JSON (lista)'
                        break
                except (ValueError, TypeError):
                    continue
            elif isinstance(valor, list):
                tipo_anidado = 'Lista'
                break
            elif isinstance(valor, dict):
                tipo_anidado = 'Diccionario'
                break
            elif isinstance(valor, (np.ndarray, pd.Series, pd.DataFrame)):
                tipo_anidado = type(valor).__name__
                break
            elif isinstance(valor, (set, tuple)):
                tipo_anidado = type(valor).__name__
                break
        
        if tipo_anidado:
            columnas_anidadas[columna] = tipo_anidado
    
    return columnas_anidadas

In [ ]:
columnas_anidadas = detectar_columnas_anidadas(movies)
for columna, tipo in columnas_anidadas.items():
    print(f"Columna '{columna}' contiene valores anidados de tipo: {tipo}")

TRATAMIENTO DE COLUMNAS ANIDADAS

BELONGS COLLECTION

In [ ]:
collection_belongs=movies['belongs_to_collection']
collection_belongs=pd.DataFrame(collection_belongs)

collection_belongs.head(2)

In [ ]:
collection_belongs.count()

In [ ]:
columnas_a_limpiarr = ['belongs_to_collection']
collection_belongs = limpiar_columnas(collection_belongs, columnas_a_limpiarr)
collection_belongs.head(2)

In [ ]:
def desanidar_y_crear_columnas(df, columnas, palabras_clave):
    # Diccionario para almacenar las nuevas columnas
    columnas_nuevas = {clave: [] for clave in palabras_clave}
    
    # Iterar sobre las columnas que contienen datos anidados
    for col in columnas:
        for index, row in df.iterrows():
            # Verificar si el valor es una cadena, de lo contrario, continuar con la siguiente fila
            if isinstance(row[col], str):
                # Separar la cadena por comas
                items = row[col].split(',')
                
                # Diccionario temporal para almacenar los valores de cada fila
                temp_dict = {clave: None for clave in palabras_clave}
                
                for item in items:
                    # Buscar coincidencias de las palabras clave seguidas del carácter ':'
                    for clave in palabras_clave:
                        if clave in item:
                            # Extraer el valor que sigue al carácter ':'
                            valor = re.search(rf'{clave}\s*:\s*(.*)', item)
                            if valor:
                                temp_dict[clave] = valor.group(1).strip()
                
                # Almacenar los valores en las listas correspondientes
                for clave in palabras_clave:
                    columnas_nuevas[clave].append(temp_dict[clave])
            else:
                # Si el valor no es una cadena, agregar None a las listas correspondientes
                for clave in palabras_clave:
                    columnas_nuevas[clave].append(None)

    # Crear un nuevo DataFrame con las nuevas columnas
    df_nuevo = pd.DataFrame(columnas_nuevas)
    
    return df_nuevo

In [ ]:
palabras_clave = ['id','name', 'poster_path', 'backdrop_path']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['belongs_to_collection']
collection_belongs= desanidar_y_crear_columnas(collection_belongs, columnas_a_desanidar, palabras_clave)

# Mostrar el nuevo DataFrame
collection_belongs.head(2)

In [ ]:
collection_belongs.count()

In [ ]:
collection_belongs.isnull().sum()

In [ ]:
collection_belongs.dropna(inplace=True)

In [ ]:
collection_belongs.isnull().sum()

In [ ]:
collection_belongs.head(2)

In [ ]:
collection_belongs.count()

GENRES

In [ ]:
genres=movies['genres']
genres=pd.DataFrame(genres)
genres.head(2)

In [ ]:
columnas_a_limpiarr = ['genres']
genres = limpiar_columnas(genres, columnas_a_limpiarr)
genres.head()

In [ ]:
palabras_clave = ['id','name']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['genres']
genres= desanidar_y_crear_columnas(genres, columnas_a_desanidar, palabras_clave)



In [ ]:
# Mostrar el nuevo DataFrame
genres.head(2)

In [ ]:
genres.count()

In [ ]:
genres.isnull().sum()

PRODUCTION COMPANIES

In [ ]:
production_companies=movies['production_companies']
production_companies=pd.DataFrame(production_companies)
production_companies.head(2)


In [ ]:
columnas_a_limpiarr = ['production_companies']
production_companies = limpiar_columnas(production_companies, columnas_a_limpiarr)
production_companies.head()


In [ ]:
palabras_clave = ['id','name']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['production_companies']
production_companies= desanidar_y_crear_columnas(production_companies, columnas_a_desanidar, palabras_clave)

# Mostrar el nuevo DataFrame
production_companies.head()

In [ ]:
production_companies.count()

PRODUCTION COUNTRIES

In [ ]:
production_countries=movies['production_countries']
production_countries=pd.DataFrame(production_countries)
production_countries.head(2)

In [ ]:
columnas_a_limpiarr = ['production_countries']
production_countries = limpiar_columnas(production_countries, columnas_a_limpiarr)
production_countries.head()

In [ ]:
palabras_clave = ['iso_3166_1','name']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['production_countries']
production_countries= desanidar_y_crear_columnas(production_countries, columnas_a_desanidar, palabras_clave)

# Mostrar el nuevo DataFrame
production_countries.head(2)

In [ ]:
production_countries.count()

 SPOKEN LANGUAGES

In [ ]:
spoken_languages=movies['spoken_languages']
spoken_languages=pd.DataFrame(spoken_languages)
spoken_languages.head(2)

In [ ]:
columnas_a_limpiarr = ['spoken_languages']
spoken_languages = limpiar_columnas(spoken_languages, columnas_a_limpiarr)
spoken_languages.head()

In [ ]:
palabras_clave = ['iso_639_1','name']

# Supongamos que tienes un DataFrame llamado df1 con columnas a desanidar
columnas_a_desanidar = ['spoken_languages']
spoken_languages= desanidar_y_crear_columnas(spoken_languages, columnas_a_desanidar, palabras_clave)

# Mostrar el nuevo DataFrame
spoken_languages.head(2)

In [ ]:
spoken_languages.count()

__________________________________________________________________________________________________________________________________________________________________________________

# **DATASETS QUE CONSUMIRAN LAS FUNCIONES DE LA API**

## MOVIES

Eliminacion de las columnas que no serán utilizadas: video, imdb_id, adult, original_title, poster_path y homepage.

In [ ]:
def eliminar_columnas(df, columnas_a_eliminar):
   
    df_nuevo = df.drop(columns=columnas_a_eliminar, errors='ignore')
    return df_nuevo


columnas_a_eliminar = ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages','video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies_nuevo= eliminar_columnas(movies, columnas_a_eliminar)

movies_nuevo.head(2)

In [ ]:
movies_nuevo.rename(columns={'id': 'id_pelicula'}, inplace=True)

In [ ]:
def merge_dataframes(df1, df2, columns_to_add, on_column, how='left'):
    """
    Une dos DataFrames y agrega solo las columnas especificadas del segundo DataFrame al primero.

    Parameters:
    - df1 (pd.DataFrame): El DataFrame principal al que se agregarán las columnas.
    - df2 (pd.DataFrame): El DataFrame del que se tomarán las columnas a agregar.
    - columns_to_add (list of str): Lista con los nombres de las columnas del segundo DataFrame que se desean agregar.
    - on_column (str): El nombre de la columna sobre la cual se realizará la unión.
    - how (str): Tipo de unión, como 'left', 'right', 'inner', o 'outer'. Por defecto es 'left'.

    Returns:
    - pd.DataFrame: DataFrame resultante con las columnas agregadas.
    """
    # Asegurarse de que las columnas de unión tengan el mismo tipo de dato
    df1[on_column] = df1[on_column].astype(str)
    df2[on_column] = df2[on_column].astype(str)
    
    # Realiza la unión entre los DataFrames
    df_merged = pd.merge(df1, df2[[on_column] + columns_to_add], on=on_column, how=how)
    return df_merged


FUNCIONES QUE VAN A CONSUMIR EL DATASET MOVIES_NUEVO

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de

In [ ]:
movies_nuevo.head(2)

In [ ]:
movies_nuevo.isnull().sum()

In [ ]:
# Eliminar la columna 'B' sin crear un nuevo DataFrame
movies_nuevo.drop('tagline', axis=1, inplace=True)

In [ ]:
# Eliminar filas con valores nulos
movies_nuevo.dropna(inplace=True)

In [ ]:
movies_nuevo.head(2)

In [ ]:
movies_nuevo.isnull().sum()

In [ ]:
movies_nuevo.count()

## CREW

FUNCION QUE VA A USAR EL DATASET CREW

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [ ]:
columns_to_add = ['title','release_date' ,'budget', 'revenue','return']
on_column = 'id_pelicula'
how = 'inner'

directores= merge_dataframes(crew, movies_nuevo, columns_to_add, on_column, how)
directores.head(2)

In [ ]:
def filtrar_directores(df):
    # Filtra las filas donde la columna 'jobs' es igual a 'Director'
    df_directores = df[df['job'] == 'Director']
    return df_directores

In [ ]:
# Aplica la función para filtrar directores
directores = filtrar_directores(directores)

In [ ]:
directores.head(3)

In [ ]:
# Eliminar filas con valores nulos
directores.dropna(inplace=True)

In [ ]:
directores.isnull().sum()

## CAST

FUNCIO QUE CONSUMIRA EL DATASET CAST

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [ ]:
columns_to_add = ['title', 'return']
on_column = 'id_pelicula'
how = 'inner'

actores= merge_dataframes(cast, movies_nuevo, columns_to_add, on_column, how)
actores.head(2)

____________________________________________________________________________________________________________________________________________________

In [ ]:
# Eliminar filas con valores nulos
actores.dropna(inplace=True)

In [ ]:
actores.isnull().sum()

__________________________________________________________________________________________________________________________________________________________________________________

## **EDA**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [ ]:
# Análisis descriptivo
movies_nuevo.describe()

In [ ]:
np.shape(movies_nuevo)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Cargar el dataset
# dataset = pd.read_csv('ruta_al_archivo.csv')



# Seleccionar solo columnas de tipo int o float
numeric_columns = movies_nuevo.select_dtypes(include=['int', 'float']).columns

# Correlación entre variables
plt.figure(figsize=(len(numeric_columns) * 2, 8))
sns.heatmap(movies_nuevo[numeric_columns].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap de Correlaciones')
plt.show()


In [ ]:
movies_nuevo[numeric_columns].corr()

In [ ]:
# Distribución de 'vote_average'
sns.histplot(movies_nuevo['vote_average'], bins=20)
plt.show()

In [ ]:
# Correlación entre 'popularity' y 'vote_average'
sns.scatterplot(x='popularity', y='vote_average', data=movies_nuevo)
plt.show()

In [ ]:

# Detección de outliers
def detectar_outliers(df, columnas):
    outliers = {}
    for columna in columnas:
        Q1 = df[columna].quantile(0.25)
        Q3 = df[columna].quantile(0.75)
        IQR = Q3 - Q1
        lim_inferior = Q1 - 1.5 * IQR
        lim_superior = Q3 + 1.5 * IQR
        outliers[columna] = df[(df[columna] < lim_inferior) | (df[columna] > lim_superior)][columna].count()
    return outliers



In [ ]:
# Lista de columnas a analizar
columnas = ['budget', 'revenue', 'return']

# Obtener el número de outliers por columna
outliers = detectar_outliers(movies_nuevo, columnas)
print("Número de outliers por columna:", outliers)

# Generar el boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=movies_nuevo[columnas])
plt.title('Boxplot de Variables')
plt.show()

## GUARDAR CSV 

In [ ]:
# Guardar el DataFrame como un archivo CSV
#movies_nuevo.to_csv('MoviesF.csv', index=False)

In [ ]:
# Guardar el DataFrame como un archivo CSV
#directores.to_csv('Directores.csv', index=False)

In [ ]:
# Guardar el DataFrame como un archivo CSV
#actores.to_csv('Actores.csv', index=False)